# Notebook Setup

In [1]:
## Import libraries
# Standard libraries
import os
import shutil
import numpy as np

# Azure ML SDK libraries
from azureml.core import Workspace, Model, Dataset, Experiment
from azureml.widgets import RunDetails
from azureml.core.resource_configuration import ResourceConfiguration
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.train.estimator import Estimator

# Azure ML Setup

We need programmatic access to the workspace

In [2]:
## Connect to Azure ML workspace using SDK
# Check core SDK version number
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK.\n")

# Load workspace
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

You are currently using version 1.0.85 of the Azure ML SDK.

Workspace name: sbazuremlws
Azure region: westeurope
Subscription id: bf088f59-f015-4332-bd36-54b988be7c90
Resource group: sbazuremlrg


# Create / Retrieve Azure ML Training Cluster Compute Target

We can either use this cluster for training or train locally on our Compute Instance (Notebook VM)

In [3]:
## Retrieve already existing training cluster or create a new one if it does not exist
# Choose a name for the AmlCompute cluster.
amlcompute_cluster_name = "cpu-cluster-1"

# Check if this compute target already exists in the workspace.
found = False
cts = ws.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'cpu-cluster-1':
    found = True
    print('Found existing compute target.')
    compute_target = cts[amlcompute_cluster_name]
    
if not found:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_DS12_V2", # for GPU, use "STANDARD_NC6"
                                                                # vm_priority = 'lowpriority', # optional
                                                                max_nodes = 6)

    # Create the cluster.
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)
    
print('Checking cluster status...')
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.

compute_target.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)

# For a more detailed view of current AmlCompute status, use get_status().

Creating a new compute target...
Checking cluster status...
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


# Copy All Relevant Files to Model_Training Folder to Give Remote Experiment Run Access

Now we need to copy all files that are needed for the training run to a separate folder. The remote compute will get all the files from this folder during training time.

In [4]:
# Create a folder for the experiment files (This has to be indicated as source directory in the Estimator Class below)
training_folder = 'model_training'
os.makedirs(training_folder, exist_ok=True)

## Copy Data

In [5]:
# Create folder "data"
os.makedirs("data", exist_ok=True)

In [6]:
# Load data from Azure ML dataset (authentication to underlying datastore via SAS token or access key if storage account
# or via service principal if data lake gen 2)
dataset_name = 'customer-churn'

customerchurn = Dataset.get_by_name(workspace=ws, name=dataset_name)

# Load the Tabular Dataset into pandas DataFrame
telcom = customerchurn.to_pandas_dataframe()

telcom.to_csv('data/customer_churn.csv', index=False, header=True)

In [7]:
# Copy the data file into the experiment folder
os.makedirs(training_folder + "/data", exist_ok=True)
shutil.copy('data/customer_churn.csv', os.path.join(training_folder, "data/customer_churn.csv"))

'model_training/data/customer_churn.csv'

## Copy Dependencies

In [8]:
# Create folder "dependencies"
os.makedirs("dependencies", exist_ok=True)

In [9]:
# Create dependencies object and store it in file
from azureml.core.environment import CondaDependencies

conda_dep = CondaDependencies()
conda_dep.add_channel("plotly")
conda_dep.add_conda_package("scikit-learn")
conda_dep.add_conda_package("statsmodels")
conda_dep.add_conda_package("plotly")
conda_dep.add_conda_package("psutil")
conda_dep.add_conda_package("plotly-orca")
conda_dep.add_conda_package("matplotlib")

conda_dep.save(path='dependencies/conda_dependencies.yaml')

'dependencies/conda_dependencies.yaml'

In [10]:
# Copy the dependencies file into the experiment folder
os.makedirs(training_folder + "/dependencies", exist_ok=True)
shutil.copy("dependencies/conda_dependencies.yaml", os.path.join(training_folder, "dependencies/conda_dependencies.yaml"))

'model_training/dependencies/conda_dependencies.yaml'

## Copy Utility Functions

In [11]:
# Copy the utility files into the experiment folder
os.makedirs(training_folder + "/utilities", exist_ok=True)
shutil.copy("utilities/preprocess_dataset.py", os.path.join(training_folder, "utilities/preprocess_dataset.py"))

'''# Copy the utilities folder into the experiment folder
try:
    shutil.copytree(src="utilities", dst=os.path.join(training_folder, "utilities/"), copy_function = shutil.copy)
except FileExistsError:
    print("The utilities folder has already been copied. Please delete it first.")'''

'# Copy the utilities folder into the experiment folder\ntry:\n    shutil.copytree(src="utilities", dst=os.path.join(training_folder, "utilities/"), copy_function = shutil.copy)\nexcept FileExistsError:\n    print("The utilities folder has already been copied. Please delete it first.")'

## Create Training Script

In [12]:
%%writefile $training_folder/customer_churn_training.py

### Setup
## Import libraries
# Import preprocessing function which is defined in utilities folder
import utilities.preprocess_dataset

# Standard libraries
import pandas as pd
import numpy as np
import joblib
import argparse
import os
import subprocess

# Azure ML libraries
from azureml.core import Run

# Data preprocessing libraries
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Model training libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import scorer, f1_score, precision_score, recall_score, plot_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
import statsmodels.api as sm

# Visualization libraries
import plotly.graph_objs as go
import plotly.subplots as sp
import plotly.offline as py
import plotly.io as pio
import plotly
import matplotlib.pyplot as plt

## Create outputs folder (to store model)
os.makedirs('outputs', exist_ok=True)

'''## Orca configuration (this is needed for plotly graphs)
pipe = subprocess.Popen("which orca", shell=True, stdout=subprocess.PIPE).stdout
orca_path = pipe.read()[1:-1].decode('UTF-8')
print("\n Orca Path \n: " + orca_path)

# Path of executable
pio.orca.config.executable = "/" + orca_path
# Configure plotly.py to run orca using Xvfb
pio.orca.config.use_xvfb = True
# Save config
pio.orca.config.save()'''

## Get the experiment run context
run = Run.get_context()

## Expose model hyperparameters and cross validation parameters as script arguments
parser = argparse.ArgumentParser()
parser.add_argument('--reg_rate_param_1', type=float, dest='reg_rate_param_1', default=[10])
parser.add_argument('--reg_rate_param_2', type=float, dest='reg_rate_param_2', default=[30])
parser.add_argument('--reg_rate_param_3', type=float, dest='reg_rate_param_3', default=[100])
parser.add_argument('--reg_rate_param_4', type=float, dest='reg_rate_param_4', default=[300])
parser.add_argument('--reg_rate_param_5', type=float, dest='reg_rate_param_5', default=[1000])
parser.add_argument('--cv_scorer', type=str, dest='cv_scorer', default='roc_auc')
parser.add_argument('--cv_folds', type=int, dest='cv_folds', default=5)
args = parser.parse_args()
reg_rate_param_1 = args.reg_rate_param_1
reg_rate_param_2 = args.reg_rate_param_2
reg_rate_param_3 = args.reg_rate_param_3
reg_rate_param_4 = args.reg_rate_param_4
reg_rate_param_5 = args.reg_rate_param_5
cv_scorer = args.cv_scorer
cv_folds = args.cv_folds

# Make a list that contains the different hyperparameters to be tested in cross validation
reg_rate_params = [reg_rate_param_1, reg_rate_param_2, reg_rate_param_3, reg_rate_param_4, reg_rate_param_5]

run.log('Cross Validation Scoring Metric', cv_scorer) # this is retrieved as script argument
run.log('Count of Cross Validation Folds', cv_folds) # this is retrieved as script argument
run.log_list('Tested Regularization Hyperparameters C', reg_rate_params) # this is retrieved as script argument

# Load the dataset
print("\n Loading Data... \n")
telcom = pd.read_csv('data/customer_churn.csv')

telcom.drop("PartitionDate", axis=1, inplace=True)

print("\n Preprocessing Data... \n")
# Preprocess dataset
X, y  = utilities.preprocess_dataset.run(telcom)

# Get categorical features
categorical_features = X.nunique()[X.nunique() < 6].keys().tolist() # get columns with less than 6 unique values

# Get numerical features
numeric_features = [x for x in X.columns if x not in categorical_features]

# Get all feature columns
features = categorical_features + numeric_features

# Create the preprocessing pipelines for both numeric and categorical features.
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehotencoder', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now this is a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(class_weight=None, dual=False, fit_intercept=True,
                                                        intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
                                                        penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
                                                        verbose=0, warm_start=False))])

parameters={"classifier__C":reg_rate_params}

# Split dataset into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Instantiate the GridSearchCV object: cv
cv = GridSearchCV(clf, parameters, scoring=cv_scorer, cv=cv_folds)

# Fit the cv pipeline
cv.fit(X_train,y_train)

# Get best parameter from cross validation
best_tested_param = cv.best_params_
print ("Best parameter for C : ", best_tested_param)
run.log("Best C parameter", best_tested_param)    

# Take the best classifier of cross validation
clf = cv.best_estimator_

# Fit the best classifier again to retrieve feature importance
clf.fit(X_train,y_train)

## Model evaluation
predictions   = clf.predict(X_test)
probabilities = clf.predict_proba(X_test)

# Create feature importance
features_after_onehot = numeric_features # initialize list with numeric features as first two coefficients are the numeric features

# Add all one-hot-encoded categorical features in the right order
for i in range(len(clf.named_steps['preprocessor'].named_transformers_["cat"].named_steps['onehotencoder'].categories_)):
    for j in clf.named_steps['preprocessor'].named_transformers_["cat"].named_steps['onehotencoder'].categories_[i]:
        features_after_onehot.append(categorical_features[i]+ "_" + j)
        
# Get coefficients from classifier
coefficients  = pd.DataFrame(clf.named_steps['classifier'].coef_.ravel())

# Create dataframe of  feature columns
column_df     = pd.DataFrame(features_after_onehot)

# Merge feature columns with coefficients
coef_sumry    = (pd.merge(coefficients,column_df,left_index= True,
                          right_index= True, how = "left"))

coef_sumry.columns = ["coefficients", "features"]

# Sort coefficients according to size
coef_sumry    = coef_sumry.sort_values(by = "coefficients",ascending = False)

# Calculate evaluation metrics
class_report = classification_report(y_test,predictions)
class_metrics = precision_recall_fscore_support(y_test, predictions, beta=1.0, average="binary", pos_label=0)
accuracy = accuracy_score(y_test,predictions)
conf_matrix = confusion_matrix(y_test,predictions)
model_roc_auc = roc_auc_score(y_test,predictions)

precision = class_metrics[0]
recall = class_metrics[1]
fscore = class_metrics[2]

# Log and print model evaluation information
print (clf)

print ("\n Classification report : \n", class_report)

print ("Accuracy Score : ", accuracy)
run.log("Accuracy", np.float(accuracy))    

print ("Area Under Curve : ",model_roc_auc,"\n")
run.log("AUC", np.float(model_roc_auc))

print ("Precision : ",precision,"\n")
run.log("Precision", np.float(precision))

print ("Recall : ",recall,"\n")
run.log("Recall", np.float(recall))

print ("F Score : ",fscore,"\n")
run.log("F Score", np.float(fscore))

fpr,tpr,thresholds = roc_curve(y_test,probabilities[:,1])

# Plot confusion matrix
class_names = ["Churn", "Non Churn"]

trace1 = plot_confusion_matrix(clf, X_test, y_test,
                               display_labels=class_names,
                               cmap="inferno",
                               normalize="true")

trace1.ax_.set_title("Normalized Confusion Matrix")
trace1.ax_.grid(False)
run.log_image("Normalized Confusion Matrix", plot=plt)

plt.clf()

# Plot roc curve
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % model_roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
run.log_image("Receiver Operating Characteristic", plot=plt)

plt.clf()

# Plot feature importance
x = coef_sumry["features"]
y = coef_sumry["coefficients"]

mask1 = y >= 0
mask2 = y < 0

objects = coef_sumry["features"]
x_pos = np.arange(len(coef_sumry["features"]))

plt.bar(x[mask1], y[mask1], color = 'green')
plt.bar(x[mask2], y[mask2], color = 'red')
plt.xticks(x_pos, x, rotation='vertical')
plt.yticks(rotation='horizontal')
plt.ylabel('Feature Coefficient')
plt.title('Feature Importance')
plt.tight_layout()
run.log_image("Feature Importance", plot=plt)
    
plt.clf()

# Save the trained model in the outputs folder
joblib.dump(value=clf, filename='outputs/customer_churn_log_reg_model.pkl')

run.complete()

Overwriting model_training/customer_churn_training.py


# Use an Estimator to Run the Script as an Experiment

The estimator needs a source directory where it gets all files relevant for training.
An entry script (training script) must be specified as well as the variables that have
been parameterized in the training script (here the parameters to be tested for the C parameter of LogisticRegression).
Also, a dependencies file with all the libraries need for model training has to be specified to build the docker image.
The training can be done locally or with the create remote compute target.

In [13]:
estimator_training_mode = 'local' # can be replaced by compute_target

# Create an estimator
estimator = Estimator(source_directory=training_folder,
                      entry_script='customer_churn_training.py',
                      script_params = {'--reg_rate_param_1': 100,
                                       '--reg_rate_param_2': 300,
                                       '--reg_rate_param_3': 10,
                                       '--reg_rate_param_4': 30,
                                       '--reg_rate_param_5': 1000,
                                       '--cv_scorer': "roc_auc",
                                       '--cv_folds': 5},
                      compute_target= estimator_training_mode,
                      conda_dependencies_file = 'dependencies/conda_dependencies.yaml'
                      )

# Create an experiment
experiment_name = 'customer-churn-log-reg-experiment'
experiment = Experiment(workspace = ws, name = experiment_name)

# Run the experiment based on the estimator
run = experiment.submit(config=estimator)
run.wait_for_completion(show_output=True)

RunId: customer-churn-log-reg-experiment_1585980078_7ccf0f9f
Web View: https://ml.azure.com/experiments/customer-churn-log-reg-experiment/runs/customer-churn-log-reg-experiment_1585980078_7ccf0f9f?wsid=/subscriptions/bf088f59-f015-4332-bd36-54b988be7c90/resourcegroups/sbazuremlrg/workspaces/sbazuremlws

Streaming azureml-logs/70_driver_log.txt

Starting the daemon thread to refresh tokens in background for process with pid = 9
Entering Run History Context Manager.
Preparing to call script [ customer_churn_training.py ] with arguments: ['--reg_rate_param_1', '100', '--reg_rate_param_2', '300', '--reg_rate_param_3', '10', '--reg_rate_param_4', '30', '--reg_rate_param_5', '1000', '--cv_scorer', 'roc_auc', '--cv_folds', '5']
After variable expansion, calling script [ customer_churn_training.py ] with arguments: ['--reg_rate_param_1', '100', '--reg_rate_param_2', '300', '--reg_rate_param_3', '10', '--reg_rate_param_4', '30', '--reg_rate_param_5', '1000', '--cv_scorer', 'roc_auc', '--cv_fold

{'runId': 'customer-churn-log-reg-experiment_1585980078_7ccf0f9f',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-04-04T06:01:22.22988Z',
 'endTimeUtc': '2020-04-04T06:01:34.888675Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '641166df-0264-434f-bffb-e088dc4fb155',
  'azureml.git.repository_uri': 'https://github.com/sebastianbirk/customer-churn-prediction-azure-ml.git',
  'mlflow.source.git.repoURL': 'https://github.com/sebastianbirk/customer-churn-prediction-azure-ml.git',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': '6ea94434c5f95819c86416341d818dcc1c92eb77',
  'mlflow.source.git.commit': '6ea94434c5f95819c86416341d818dcc1c92eb77',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [],
 'runDefinition': {'script': 'customer_churn_training.py',
  'useAbsolutePath': False,
  'arguments': ['--reg_rate_param_1',
   '100',
   '--reg_rate_param_2',
   '300',
   '--reg_rate_param_3',


In [14]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

# Register the Trained Model

In [15]:
# Register the model
run.register_model(model_path='outputs/customer_churn_log_reg_model.pkl', model_name='customer_churn_log_reg_model',
                   tags={'Training Context':'Estimator', 'Algorithm':'Logistic Regression'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy'],
                              'Precision': run.get_metrics()['Precision'], 'Recall': run.get_metrics()['Recall'],
                              'F Score': run.get_metrics()['F Score']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

customer_churn_log_reg_model version: 5
	 Training Context : Estimator
	 Algorithm : Logistic Regression
	 AUC : 0.7273229549221619
	 Accuracy : 0.8161816891412349
	 Precision : 0.8599640933572711
	 Recall : 0.9029217719132894
	 F Score : 0.8809195402298852


customer_churn_log_reg_model version: 4
	 Training Context : Estimator
	 Algorithm : Logistic Regression
	 AUC : 0.7032768315318266
	 Accuracy : 0.7998580553584103
	 Precision : 0.8504923903312444
	 Recall : 0.892018779342723
	 F Score : 0.8707607699358386


AutoML0b6c959a634 version: 1


customer_churn_dt_model version: 1
	 Training Context : Notebook
	 Algorithm : Decision Tree


customer_churn_log_reg_model version: 3
	 Training Context : Estimator
	 Algorithm : Logistic Regression
	 AUC : 0.7128652597402597
	 Accuracy : 0.8005677785663591
	 Precision : 0.8337825696316262
	 Recall : 0.90625
	 F Score : 0.8685072531586336


customer_churn_log_reg_model version: 2
	 Training Context : Estimator
	 Algorithm : Logistic Regression
	

# Appendix

Old script:

%%writefile $training_folder/customer_churn_training_old.py

### Setup
## Import libraries
# Standard libraries
import pandas as pd
import numpy as np
import joblib
import argparse
import os
import subprocess

# Azure ML libraries
from azureml.core import Run

# Data preprocessing libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# Model training libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import scorer, f1_score, precision_score, recall_score, plot_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
import statsmodels.api as sm

# Visualization libraries
import plotly.graph_objs as go
import plotly.subplots as sp
import plotly.offline as py
import plotly.io as pio
import plotly
import matplotlib.pyplot as plt

# Create outputs folder
os.makedirs('outputs', exist_ok=True)

'''## Orca configuration
pipe = subprocess.Popen("which orca", shell=True, stdout=subprocess.PIPE).stdout
orca_path = pipe.read()[1:-1].decode('UTF-8')
print("\n Orca Path \n: " + orca_path)

# Path of executable
pio.orca.config.executable = "/" + orca_path
# Configure plotly.py to run orca using Xvfb
pio.orca.config.use_xvfb = True
# Save config
pio.orca.config.save()'''

# Get the experiment run context
run = Run.get_context()

# Load the dataset
print("\n Loading Data... \n")
telcom = pd.read_csv('customer_churn.csv')

# Preprocess dataset
telcom = preprocess_dataset(telcom)

# Scaling of numerical columns
std = StandardScaler()
scaled = std.fit_transform(telcom[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)

# Dropping original values and merging scaled values for numerical columns
telcom = telcom.drop(columns = num_cols,axis = 1)
telcom = telcom.merge(scaled,left_index=True,right_index=True,how = "left")


### Model Training
# Split data into train and test
train,test = train_test_split(telcom, test_size = .25 ,random_state = 42)
    
# Separate dependent and independent variables and exclude ID column
cols    = [i for i in telcom.columns if i not in Id_col + target_col]
train_X = train[cols]
train_Y = train[target_col]
test_X  = test[cols]
test_Y  = test[target_col]

# Write a function for generic model training    
def telecom_churn_prediction(algorithm,training_x,testing_x,
                             training_y,testing_y,cols,cf):
    '''
    Function signature
    algorithm     - algorithm used 
    training_x    - predictor variables dataframe (training)
    testing_x     - predictor variables dataframe (testing)
    training_y    - target variable (training)
    training_y    - target variable (testing)
    cols          - features
    cf - ["coefficients","features"] (cooefficients for logistic regression, features for tree based models)
    '''
    
    # Fit the model
    algorithm.fit(training_x,training_y)
    predictions   = algorithm.predict(testing_x)
    probabilities = algorithm.predict_proba(testing_x)
    
    # Store coefficients
    if   cf == "coefficients" :
        coefficients  = pd.DataFrame(algorithm.coef_.ravel())
    elif cf == "features" :
        coefficients  = pd.DataFrame(algorithm.feature_importances_)
        
    column_df     = pd.DataFrame(cols)
    coef_sumry    = (pd.merge(coefficients,column_df,left_index= True,
                              right_index= True, how = "left"))
    coef_sumry.columns = ["coefficients","features"]
    coef_sumry    = coef_sumry.sort_values(by = "coefficients",ascending = False)
    
    # Evaluate the model
    class_report = classification_report(testing_y,predictions)
    class_metrics = precision_recall_fscore_support(testing_y, predictions, beta=1.0, average="binary", pos_label=0)
    accuracy = accuracy_score(testing_y,predictions)
    conf_matrix = confusion_matrix(testing_y,predictions)
    model_roc_auc = roc_auc_score(testing_y,predictions)
    
    precision = class_metrics[0]
    recall = class_metrics[1]
    fscore = class_metrics[2]
    
    # Log and print model evaluation information
    print (algorithm)
    
    print ("\n Classification report : \n", class_report)
    
    print ("Accuracy Score : ", accuracy)
    run.log("Accuracy", np.float(accuracy))    
    
    print ("Area Under Curve : ",model_roc_auc,"\n")
    run.log("AUC", np.float(model_roc_auc))
    
    print ("Precision : ",precision,"\n")
    run.log("Precision", np.float(precision))
    
    print ("Recall : ",recall,"\n")
    run.log("Recall", np.float(recall))
    
    print ("F Score : ",fscore,"\n")
    run.log("F Score", np.float(fscore))
    
    fpr,tpr,thresholds = roc_curve(testing_y,probabilities[:,1])
    
    # Plot confusion matrix
    class_names = ["Churn", "Non Churn"]
    
    trace1 = plot_confusion_matrix(algorithm, testing_x, testing_y,
                                   display_labels=class_names,
                                   cmap="inferno",
                                   normalize="true")
    
    trace1.ax_.set_title("Normalized Confusion Matrix")
    trace1.ax_.grid(False)
    run.log_image("Normalized Confusion Matrix", plot=plt)
    
    plt.clf()
    
    # Plot roc curve
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % model_roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    run.log_image("Receiver Operating Characteristic", plot=plt)
    
    plt.clf()
    
    # Plot feature importance
    x = coef_sumry["features"]
    y = coef_sumry["coefficients"]

    mask1 = y >= 0
    mask2 = y < 0

    objects = coef_sumry["features"]
    x_pos = np.arange(len(coef_sumry["features"]))

    plt.bar(x[mask1], y[mask1], color = 'green')
    plt.bar(x[mask2], y[mask2], color = 'red')
    plt.xticks(x_pos, x, rotation='vertical')
    plt.yticks(rotation='horizontal')
    plt.ylabel('Feature Coefficient')
    plt.title('Feature Importance')
    plt.tight_layout()
    run.log_image("Feature Importance", plot=plt)
    
    plt.clf()
    
    '''# Plot confusion matrix
    trace1 = go.Heatmap(z = conf_matrix ,
                        x = ["Not churn","Churn"],
                        y = ["Not churn","Churn"],
                        showscale  = False,colorscale = "Picnic",
                        name = "matrix")
    
    # Plot roc curve
    trace2 = go.Scatter(x = fpr,y = tpr,
                        name = "Roc : " + str(model_roc_auc),
                        line = dict(color = ('rgb(22, 96, 167)'),width = 2))
    trace3 = go.Scatter(x = [0,1],y=[0,1],
                        line = dict(color = ('rgb(205, 12, 24)'),width = 2,
                        dash = 'dot'))
    
    # Plot coefficients
    trace4 = go.Bar(x = coef_sumry["features"],y = coef_sumry["coefficients"],
                    name = "coefficients",
                    marker = dict(color = coef_sumry["coefficients"],
                                  colorscale = "Picnic",
                                  line = dict(width = .6,color = "black")))
    
    # Create subplots
    fig = sp.make_subplots(rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]],
                            subplot_titles=('Confusion Matrix',
                                            'Receiver operating characteristic',
                                            'Feature Importances'))
    
    fig.append_trace(trace1,1,1)
    fig.append_trace(trace2,1,2)
    fig.append_trace(trace3,1,2)
    fig.append_trace(trace4,2,1)
    
    # Plot layout
    fig['layout'].update(showlegend=False, title="Model performance" ,
                         autosize = False,height = 900,width = 800,
                         plot_bgcolor = 'rgba(240,240,240, 0.95)',
                         paper_bgcolor = 'rgba(240,240,240, 0.95)',
                         margin = dict(b = 195))
    fig["layout"]["xaxis2"].update(dict(title = "false positive rate"))
    fig["layout"]["yaxis2"].update(dict(title = "true positive rate"))
    fig["layout"]["xaxis3"].update(dict(showgrid = True,tickfont = dict(size = 10),
                                        tickangle = 90))
    
    image_path = "outputs/log_reg_graphs.png"
    fig.write_image(image_path)
    
    # Upload the file explicitly into artifacts 
    run.upload_file(name = image_path, path_or_stream = image_path)'''
    
    return algorithm
        
# Build logistic regression model

# Expose regularization hyperparameter as script argument
parser = argparse.ArgumentParser()
parser.add_argument('--reg_rate', type=float, dest='reg', default=0.01)
args = parser.parse_args()
reg = args.reg

run.log('Regularization Hyperparameter Lambda', np.float(reg)) # this is retrieved as script argument

logit = LogisticRegression(C=1/reg, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

logit_fitted = telecom_churn_prediction(logit,train_X,test_X,train_Y,test_Y,
                         cols,"coefficients")

# Save the trained model in the outputs folder
joblib.dump(value=logit_fitted, filename='outputs/customer_churn_log_reg_model.pkl')

run.complete()